In [1]:
import socket
import pickle
import pandas as pd
import numpy as np
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives.ciphers.aead import AESGCM
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD


In [2]:
dataset_path = 'heart3ex.csv'
data = pd.read_csv(dataset_path)
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

In [ ]:
def build_model(input_dim):
    model = Sequential([
        Dense(16, activation='relu', input_shape=(input_dim,)),
        Dense(8, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=SGD(learning_rate=0.01), loss='binary_crossentropy', metrics=['accuracy'])
    return model


In [4]:
server_ip = 'localhost'
server_port = 5000
client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
client_socket.connect((server_ip, server_port))

In [5]:
client_private_key = ec.generate_private_key(ec.SECP256R1())
client_public_key = client_private_key.public_key()

In [6]:
server_public_key_pem = client_socket.recv(1024)
server_public_key = serialization.load_pem_public_key(server_public_key_pem)
client_socket.send(client_public_key.public_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PublicFormat.SubjectPublicKeyInfo
))

178

In [7]:
from sklearn.feature_selection import mutual_info_classif
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

lasso = Lasso(alpha=0.01, max_iter=1000)
lasso.fit(X_scaled, y)
lasso_selected_features = np.where(np.abs(lasso.coef_) > 0)[0]

mutual_info = mutual_info_classif(X, y, random_state=42)
mutual_info_selected_features = np.argsort(mutual_info)[-8:] 


selected_features = np.unique(np.concatenate((lasso_selected_features, mutual_info_selected_features)))

X = X[:, selected_features]

print(f"Selected features indices: {selected_features}")


Selected features indices: [ 0  1  2  3  4  6  7  8  9 10 11 12]


In [ ]:
num_rounds = 30
for round_num in range(num_rounds):
    print(f"Starting round {round_num + 1}/{num_rounds}")
    
    encrypted_data = client_socket.recv(4096)
    shared_key = client_private_key.exchange(ec.ECDH(), server_public_key)
    derived_key = HKDF(algorithm=hashes.SHA256(), length=32, salt=None, info=b'handshake data').derive(shared_key)
    aesgcm = AESGCM(derived_key)
    nonce = b'\x00' * 12  # Example nonce
    data = aesgcm.decrypt(nonce, encrypted_data, None)
    global_weights = pickle.loads(data)

    model = build_model(input_dim=X.shape[1])  # Adjust input shape dynamically

    if global_weights is not None:
        model.set_weights(global_weights)

    model.fit(X, y, epochs=1, verbose=0)

    local_weights = model.get_weights()
    data = pickle.dumps(local_weights)
    encrypted_data = aesgcm.encrypt(nonce, data, None)
    
    client_socket.send(encrypted_data)
    print(f"Round {round_num + 1} complete.")

client_socket.close()
print("Federated learning complete and connection closed.")


Starting round 1/30


c:\Users\krish\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense" is incompatible with the layer: expected axis -1 of input shape to have value 13, but received input with shape (None, 12)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(None, 12), dtype=float32)
  • training=True
  • mask=None